# Financial Assistant Application Project Guide

## Overview
In this project, you will build a comprehensive financial assistant application using Langchain. This application demonstrates several advanced concepts including:
- Retrieval Augmented Generation (RAG)
- Conversational memory management
- Tool use with LangChain agents
- Database integration with SQL
- Web UI with Streamlit

The assistant will help users manage their finances, provide investment advice, analyze spending patterns, and offer financial education through multiple tools.

## Project Structure
Your project should follow this directory structure:
```
fintech_app/
├── app.py                  # Main Streamlit application
├── financial_knowledge.txt # Financial knowledge for RAG (provided)
├── data/                   # Directory for storing data
│   └── chroma_db/          # Vector database for RAG
├── utils/                  # Utility modules
│   ├── __init__.py         # Exports from utility modules 
│   ├
│   ├── rag.py              # RAG implementation
│   └── tools.py            # Custom tools for the agent
```

You'll be provided with:
- `financial_knowledge.txt` - A comprehensive file with financial information for the RAG system
- `finance_data.db` - A SQLite database with sample financial data

## Prerequisites

 Required packages (create a requirements.txt with):
   - langchain
   - langchain-openai
   - langchain-community
   - langchain-core
   - langchain-experimental
   - langchain-chroma
   - streamlit
   - yfinance
   - chromadb
   - python-dotenv
   - tavily-python
   - SQLAlchemy

API keys:
   - OpenAI API key
   - Tavily API key (for web search)

## Step 1: Setup Environment

1. Create a `.env` file in your project root with:
```
OPENAI_API_KEY=your_openai_key
TAVILY_API_KEY=your_tavily_key
```

2. Create the project directory structure as shown in the project structure section.

3. Make sure the financial_knowledge.txt file is in the main project directory.

## Step 2: Implement the RAG Manager (utils/rag.py)

The RAG (Retrieval Augmented Generation) system is a key component that will allow our financial assistant to provide accurate financial information by retrieving relevant content from our knowledge base.

### TODO 1: Create the RAGManager class

Implement a class called `RAGManager` that will handle document storage, retrieval, and RAG chain creation.

```python
class RAGManager:
    def __init__(self, persist_directory="fintech_app/data/chroma_db"):
        """Initialize the RAG manager with a vector store.
        
        Args:
            persist_directory: Directory where Chroma will persist the vector store data.
                              When this is provided, Chroma automatically persists data.
        """
        # TODO: Initialize embeddings using OpenAIEmbeddings
        # TODO: Initialize LLM using ChatOpenAI with gpt-4o-mini and low temperature
        # TODO: Create directory if it doesn't exist
        # TODO: Initialize Chroma vector store with persist_directory and embeddings
        # TODO: Set up retriever with search_kwargs={"k": 5}
```

**Hints**:
- Use `OpenAIEmbeddings()` to create the embeddings object
- Initialize LLM with `ChatOpenAI(model="gpt-4o-mini", temperature=0.2)`
- Create directories with `os.makedirs(os.path.dirname(persist_directory), exist_ok=True)`
- Initialize Chroma with `Chroma(persist_directory=persist_directory, embedding_function=self.embeddings)`
- Create the retriever with `self.vector_store.as_retriever(search_kwargs={"k": 5})`
- Using `persist_directory` with Chroma means data is automatically persisted without needing explicit calls

### TODO 2: Implement document addition methods

Implement methods to add documents to the vector store from files and direct text input.

```python
def add_document_from_file(self, file_path):
    """Load a document from a file and add it to the vector store.
    
    Supports both PDF and TXT files.
    
    Args:
        file_path: Path to the file (PDF or TXT)
        
    Returns:
        str: Success or error message
    """
    # TODO: Determine file type using pathlib and use appropriate loader
    # TODO: Load the document
    # TODO: Split the document into chunks using RecursiveCharacterTextSplitter
    # TODO: Add the chunks to the vector store
    # TODO: Return success message with number of chunks added
    # TODO: Handle exceptions with appropriate error messages
    
def add_text(self, text, metadata=None):
    """Add a text string directly to the vector store."""
    # TODO: Create a document from the text with metadata
    # TODO: Split the document into chunks
    # TODO: Add chunks to vector store
    # TODO: Return success message
    # TODO: Handle exceptions


**Hints**:
- For `add_document_from_file`:
  - Use `pathlib.Path(file_path).suffix.lower()` to get the file extension
  - For PDFs use `PyPDFLoader(file_path)`, for TXT use `TextLoader(file_path)`
  - Split documents with `RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)`
  - Add to vector store with `self.vector_store.add_documents(splits)`
  - Use try/except to catch and report errors

- For `add_text`:
  - Create a Document with `Document(page_content=text, metadata=metadata or {"source": "user_input"})`
  - Use the same text splitter as in `add_document_from_file`
  - Also add the splits to the vector store



### TODO 3: Implement basic RAG chain creation

Implement a method to create a RAG chain with conversation history awareness.

```python
def create_rag_chain(self):
    """Create a RAG chain for answering financial questions with conversation history support."""
    # TODO: Create a prompt for contextualizing questions based on chat history
    # TODO: Create a contextualize question prompt template
    # TODO: Create a history-aware retriever
    # TODO: Create a final response prompt template
    # TODO: Create document chain
    # TODO: Create and return the full retrieval chain
```

**Hints**:
- For contextualizing questions prompt:
  ```python
  contextualize_q_prompt = """Given above chat history and the below latest user question
      which might reference context in the chat history,
      formulate a standalone question which can be understood
      without the chat history. Do NOT answer the question,
      just reformulate it if needed and otherwise return it as is.
      Below is the latest question:

      {input}
      """
  ```

- Create prompt template with:
  ```python
  contextualize_q_prompt_template = ChatPromptTemplate.from_messages([
      MessagesPlaceholder(variable_name="chat_history"),
      ("human", contextualize_q_prompt),
  ])
  ```

- Create history-aware retriever with:
  ```python
  history_aware_retriever = create_history_aware_retriever(
      self.llm, 
      self.retriever, 
      contextualize_q_prompt_template
  )
  ```

- Create response prompt with:
  ```python
  qa_prompt = ChatPromptTemplate.from_messages([
      ("system", "You are a financial advisor. Answer the question based on the context provided. If you don't know the answer, just say 'I don't know'."),
      MessagesPlaceholder(variable_name="chat_history"),
      ("human", "{input}"),
      ("human", "Context: {context}")
  ])
  ```

- Create document chain with `create_stuff_documents_chain(self.llm, qa_prompt)`
- Create retrieval chain with `create_retrieval_chain(history_aware_retriever, document_chain)`

### TODO 4: Implement conversational RAG chain

Implement a method that creates a RAG chain with automatic message history management.

```python
def get_conversational_rag_chain(self):
    """Creates a conversational RAG chain with automatic message history management.
    
    Returns:
        A runnable chain that can be used with session IDs to maintain conversation history.
    """
    # TODO: Import necessary components for chat history management
    # TODO: Create a store to track conversation histories
    # TODO: Create a function to get or create a new chat history for a session
    # TODO: Create the basic RAG chain
    # TODO: Wrap it with message history management and return it
```

**Hints**:
- Import required classes:
  ```python
  from langchain_community.chat_message_histories import ChatMessageHistory
  from langchain_core.chat_history import BaseChatMessageHistory
  from langchain_core.runnables.history import RunnableWithMessageHistory
  ```

- Create store and session history function:
  ```python
  store = {}
  
  def get_session_history(session_id: str) -> BaseChatMessageHistory:
      if session_id not in store:
          store[session_id] = ChatMessageHistory()
      return store[session_id]
  ```

- Create the basic RAG chain with `rag_chain = self.create_rag_chain()`

- Wrap with message history:
  ```python
  conversational_rag_chain = RunnableWithMessageHistory(
      rag_chain,
      get_session_history,
      input_messages_key="input",
      history_messages_key="chat_history",
      output_messages_key="answer",
  )
  ```

- We're using this approach to maintain separate conversation histories for different users in the application

## Step 3: Implement Custom Tools (utils/tools.py)

The tools module will provide the agent with capabilities to retrieve information, perform calculations, and interact with external systems.

### TODO 1: Create the setup_tools function framework

```python
import yfinance as yf
from langchain_core.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

from .rag import RAGManager
from typing import List, cast, Optional

def setup_tools(rag_manager: RAGManager, llm, user_email: Optional[str] = None):
    """Set up the tools for the finance agent.
    
    Args:
        rag_manager: The RAG manager for financial knowledge retrieval
        llm: The language model to use for SQL toolkit
        user_email: The email of the currently logged in user
    """
    
    # TODO: Implement stock price tool function
    # TODO: Implement financial knowledge retrieval function
    # TODO: Initialize Python REPL tool
    # TODO: Initialize Tavily search tool
    # TODO: Create the tools list with all tools
    # TODO: Get SQL toolkit and add its tools
    # TODO: Return the complete tools list
```

**Hints**:
- Importing `cast` from typing is necessary for type checking when extending the tools list
- `Optional[str]` is used for the user_email parameter since it might not be available at initialization
- The function signature includes the RAG manager and LLM since they're needed for different tools

### TODO 2: Implement the stock price tool

```python
# TODO: Implement this function within setup_tools
def get_stock_price(ticker):
    """Get the latest price for a stock ticker."""
    # TODO: Use yfinance to get stock data
    # TODO: Return formatted price information
    # TODO: Handle exceptions
```

**Hints**:
- Use this implementation:
  ```python
  def get_stock_price(ticker):
      """Get the latest price for a stock ticker."""
      try:
          stock = yf.Ticker(ticker)
          price = stock.history(period="1d")['Close'].iloc[-1]
          return f"The current price of {ticker} is ${price:.2f}"
      except Exception as e:
          return f"Error fetching stock price: {str(e)}"
  ```

- This function uses yfinance to fetch the latest closing price for a given stock symbol
- We format the response as a user-friendly string with proper currency formatting
- The try/except block ensures errors are handled gracefully

### TODO 3: Implement the financial knowledge retrieval tool

```python
# TODO: Implement this function within setup_tools
def retrieve_financial_knowledge(query):
    """Retrieve financial knowledge from the vector store."""
    # TODO: Print debugging information
    # TODO: Get RAG chain with user_email as session_id
    # TODO: Invoke the chain with the query
    # TODO: Return the answer
    # TODO: Handle exceptions
```

**Hints**:
- Use this implementation:
  ```python
  def retrieve_financial_knowledge(query):
      """Retrieve financial knowledge from the vector store."""
      try:
          print(f"Retrieving financial knowledge for user: {user_email}")
          print(f"Query: {query}")
          print("="*100)
          # Use RAG chain with user_email as the session_id for conversation history
          rag_chain = rag_manager.get_conversational_rag_chain()
          response = rag_chain.invoke(
              {"input": query},
              config={"configurable": {"session_id": user_email if user_email else "anonymous"}}
          )
          
          return response["answer"]
      except Exception as e:
          return f"Error retrieving financial knowledge: {str(e)}"
  ```

- This function gets a conversational RAG chain from our manager
- We use the user's email as the session ID to maintain continuity in conversations
- The debug prints help with troubleshooting and understanding the flow
- We return the "answer" key from the response, which contains the generated content

### TODO 4: Set up the tools list

```python
# TODO: Implement within setup_tools after the function definitions
# Initialize Python REPL Tool for data analysis
python_repl = PythonREPLTool()

# Tavily Search Tool for market research
tavily_search = TavilySearchResults(max_results=3)

# Create the tools list
tools = [
    # TODO: Create Tool for stock price
    # TODO: Create Tool for Python calculator
    # TODO: Create Tool for market research
    # TODO: Create Tool for financial knowledge retrieval
]
```

**Hints**:
- For the stock price tool:
  ```python
  Tool(
      name="get_stock_price",
      func=get_stock_price,
      description="Get the current price of a stock. Input should be a valid stock ticker symbol (e.g., AAPL, MSFT)."
  )
  ```

- For the Python calculator tool:
  ```python
  Tool(
      name="python_calculator",
      func=python_repl.run,
      description="Useful for performing calculations, data analysis, or generating visualizations. Input should be Python code."
  )
  ```

- For the market research tool:
  ```python
  Tool(
      name="market_research",
      func=tavily_search.invoke,
      description="Search the web for financial news, market analysis, or investment advice. Input should be a search query."
  )
  ```

- For the financial knowledge retrieval tool:
  ```python
  Tool.from_function(
      func=retrieve_financial_knowledge,
      name="retrieve_financial_knowledge",
      description="""Retrieve financial knowledge, investment fundamentals 
      and advice from our knowledge base with conversation memory. 
      This tool maintains conversation history for contextual follow-up questions. 
      Use this for questions about financial advice, investment strategies, best practices, 
      recommendations, or when you need expert financial guidance.""",
      return_direct=True
  )
  ```

- Notice we use `return_direct=True` for the knowledge retrieval tool because we want its output to go directly to the user

### TODO 5: Add SQL Tools and complete the function

```python
# TODO: Continue implementation within setup_tools
# Get the SQL toolkit and add its tools with custom descriptions
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

def get_db_toolkit(llm: ChatOpenAI) -> SQLDatabaseToolkit:
    """Get SQL Database toolkit with pre-configured tools for the finance database.
    
    Args:
        llm: The language model to use for the toolkit
        
    Returns:
        A SQLDatabaseToolkit configured for the finance database
    """
    db = SQLDatabase.from_uri("sqlite:///fintech_app/data/finance_data.db")
    return SQLDatabaseToolkit(db=db, llm=llm) 

sql_toolkit = get_db_toolkit(llm)
sql_tools = sql_toolkit.get_tools()

# Customize database tool descriptions
for tool in sql_tools:
    # TODO: Customize sql_db_query description
    # TODO: Customize sql_db_schema description
    # TODO: Customize sql_db_list_tables description

# Add SQL tools to the list
# TODO: Extend tools list with SQL tools

# Return the complete tools list
return tools
```

**Hints**:
- For customizing the SQL query tool:
  ```python
  if tool.name == "sql_db_query":
      tool.description = """
Execute SQL queries on the finance database to retrieve information about:
1. User details (from the 'users' table) - Access user profile information
2. Transaction history (from the 'transactions' table) - Get spending history, income, expenses by category
3. Investment portfolio (from the 'portfolio' table) - Access stock holdings, purchase history, and portfolio composition

When querying for personal data, always filter using the email_id column to ensure data privacy.
Example: SELECT * FROM transactions WHERE email_id = '<user_email>'

Use this for direct SQL database queries to analyze financial data or retrieve specific information.
"""
  ```

- For customizing the SQL schema tool:
  ```python
  elif tool.name == "sql_db_schema":
      tool.description = """
Get schema information about the finance database tables:
- users: User account information (email_id, name, join_date)
- transactions: Financial transaction records (id, email_id, date, amount, category, description)
- portfolio: Investment holdings (id, email_id, symbol, shares, purchase_price, purchase_date)

Use this to understand database structure before querying.
"""
  ```

- For customizing the list tables tool:
  ```python
  elif tool.name == "sql_db_list_tables":
      tool.description = """
List all tables in the finance database 
Use this when you need to check what tables are available for querying.
"""
  ```

- Extend the tools list with:
  ```python
  tools.extend(cast(List[Tool], sql_tools))
  ```

- The `cast()` function is used to tell the type checker that sql_tools can be treated as a List of Tool objects

## Step 4: Create the Package Structure

### TODO: Create utils/__init__.py

Create a simple `__init__.py` file to export the necessary functions:

```python

from .rag import RAGManager
from .tools import setup_tools
```

**Hints**:
- This file allows you to import directly from the utils package
- For example: `from utils import RAGManager` instead of `from utils.rag import RAGManager`
- This should be created after implementing the RAG and tools modules

## Step 5: Build the Main Application (app.py)

### TODO 1: Import necessary libraries and set up the environment

```python
import streamlit as st
import os
import uuid

from tempfile import NamedTemporaryFile
from datetime import datetime

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec,RunnableLambda
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Import project modules
from utils import setup_database
from utils import RAGManager
from utils import setup_tools

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Create necessary directories
os.makedirs("fintech_app/data", exist_ok=True)

# Configure page
st.set_page_config(page_title="Siva Fintech Assistant", page_icon="💰", layout="wide")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini")
```

**Hints**:
- We're importing several LangChain modules for different parts of the application:
  - `ChatPromptTemplate` and `MessagesPlaceholder` for creating prompts
  - `RunnableWithMessageHistory` for conversation management
  - `ConfigurableFieldSpec` for configuring the runnable
  - `ChatMessageHistory` for storing conversation messages
  - `ChatOpenAI` for the language model
  - `AgentExecutor` and `create_tool_calling_agent` for agent creation
- The `uuid` module is used to generate unique IDs for conversations
- `NamedTemporaryFile` is used for handling file uploads
- We're loading the modules from our utils package

### TODO 2: Initialize session state

```python
# Initialize session state
if "store" not in st.session_state:
    st.session_state.store = {}

if "user_id" not in st.session_state:
    st.session_state.user_id = ""  

if "conversations" not in st.session_state:
    st.session_state.conversations = {}

if "conversation_counter" not in st.session_state:
    st.session_state.conversation_counter = 1

if "current_conversation_id" not in st.session_state:
    new_conv_id = str(uuid.uuid4())
    st.session_state.conversations[new_conv_id] = {
        "number": st.session_state.conversation_counter,
        "messages": []
    }
    st.session_state.current_conversation_id = new_conv_id

if "role" not in st.session_state:
    st.session_state.role = "user"
    
if "user_email" not in st.session_state:
    st.session_state.user_email = ""
    
if "email_submitted" not in st.session_state:
    st.session_state.email_submitted = False
```

**Hints**:
- `store` will contain our chat message histories
- `user_id` will store the identified user
- `conversations` is a dictionary that tracks all conversations
- `conversation_counter` keeps track of the conversation numbers
- `current_conversation_id` tracks which conversation is active
- We initialize a new conversation with a UUID and store it in the conversations dictionary
- `role` determines whether we show the user or admin interface
- `user_email` and `email_submitted` manage the user identification process

### TODO 3: Implement chat history management functions

```python
# Chat message history management
def get_session_history(user_id: str, conversation_id: str) -> ChatMessageHistory:
    # TODO: Get or create a new chat history for the user/conversation pair

def create_new_conversation():
    # TODO: Increment conversation counter
    # TODO: Create a new conversation with UUID
    # TODO: Set it as the current conversation

def switch_conversation(conv_id):
    # TODO: Set the current conversation ID
```

**Hints**:
- For `get_session_history`:
  ```python
  if (user_id, conversation_id) not in st.session_state.store:
      st.session_state.store[(user_id, conversation_id)] = ChatMessageHistory()
  return st.session_state.store[(user_id, conversation_id)]
  ```

- For `create_new_conversation`:
  ```python
  st.session_state.conversation_counter += 1
  new_conv_id = str(uuid.uuid4())
  st.session_state.conversations[new_conv_id] = {
      "number": st.session_state.conversation_counter,
      "messages": []
  }
  st.session_state.current_conversation_id = new_conv_id
  ```

- For `switch_conversation`:
  ```python
  st.session_state.current_conversation_id = conv_id
  ```

- We use a tuple of (user_id, conversation_id) as the key in our store to maintain separate histories
- Each new conversation gets a unique UUID and an incremented number for display
- The conversations dictionary stores both the number and the message history

### TODO 4: Implement chat input handling function

```python
def handle_chat_input(prompt, agent_executor_with_history):
    # TODO: Add message to current conversation
    # TODO: Configure the agent with user_id and conversation_id
    # TODO: Create a chat message with thinking indicator
    # TODO: Invoke the agent and display the response
    # TODO: Add assistant response to conversation history
    # TODO: Handle exceptions
```

**Hints**:
- Add message to current conversation with:
  ```python
  current_conv = st.session_state.conversations[st.session_state.current_conversation_id]
  current_conv["messages"].append({"role": "user", "content": prompt})
  ```

- Configure the agent with:
  ```python
  config = {
      "configurable": {
          "user_id": st.session_state.user_id,
          "conversation_id": st.session_state.current_conversation_id
      }
  }
  ```

- Create a chat message with:
  ```python
  with st.chat_message("assistant", avatar="💰"):
      response_container = st.empty()
      response_container.markdown("Thinking...")
  ```

- Invoke the agent and display the response with:
  ```python
  try:
      result = agent_executor_with_history.invoke(
          {"input": prompt},
          config=config,
      )
      
      response_container.markdown(result["output"])
      current_conv["messages"].append({"role": "assistant", "content": result["output"]})
  except Exception as e:
      response_container.markdown(f"Error: {str(e)}")
      current_conv["messages"].append({"role": "assistant", "content": f"Error: {str(e)}"})
  ```

- We use empty containers to update the response as it's generated
- We store messages in the conversation history to display them when switching conversations

### TODO 5: Implement agent setup function

```python
def setup_agent(tools):
    # TODO: Define agent system message with user_email context
    # TODO: Create prompt template with system message, history and input
    # TODO: Create agent using tool calling approach
    # TODO: Create agent executor
    # TODO: Create a combined runnable with message history
    # TODO: Return the agent executor with history
```

**Hints**:
- Define system message with:
  ```python
  system_message = f"""You are a personal finance assistant for user with email {st.session_state.user_email}.
When querying the database for user data, always filter results using:
WHERE email_id = '{st.session_state.user_email}'

You have access to tools to help with financial queries. Use these tools to provide accurate and helpful responses.

For each request:
1. Analyze what the user is asking for
2. Choose the appropriate tool to use
3. Use the tool to get the information needed
4. Present the information in a clear, educational way

When users ask for financial advice, investment recommendations, or best practices, use the retrieve_financial_knowledge tool to get relevant information from our knowledge base.

Always be helpful, clear, and educational in your responses. Explain financial concepts simply.
When providing investment advice, always include disclaimers about risk.
Never make up information - if you don't know or need more data, say so.
"""
  ```

- Create prompt template with:
  ```python
  prompt = ChatPromptTemplate.from_messages([
      ("system", system_message),
      MessagesPlaceholder(variable_name="chat_history"),
      ("human", "{input}"),
      MessagesPlaceholder(variable_name="agent_scratchpad")
  ])
  ```

- Create agent and executor with:
  ```python
  agent = create_tool_calling_agent(llm, tools, prompt)
  agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
  ```

- Create runnable with message history:
  ```python
  agent_executor_with_history = RunnableWithMessageHistory(
      agent_executor,
      get_session_history,
      input_messages_key="input",
      history_messages_key="chat_history",
      history_factory_config=[
          ConfigurableFieldSpec(
              id="user_id",
              annotation=str,
              name="User ID",
          ),
          ConfigurableFieldSpec(
              id="conversation_id",
              annotation=str,
              name="Conversation ID",
          ),
      ],
  )
  ```

- The system message includes the user's email to personalize responses and ensure data is filtered correctly
- The `agent_scratchpad` is used to show the agent's reasoning process
- The `history_factory_config` specifies what parameters are needed to get the right chat history

### TODO 6: Implement role setting function

```python
def set_role(role):
    # TODO: Set the role in session state
    # TODO: Reset email and user_id when switching to user role
```

**Hints**:
- Implement with:
  ```python
  st.session_state.role = role
  # Reset email and user_id when switching roles
  if role == "user":
      st.session_state.email_submitted = False
      st.session_state.user_email = ""
      st.session_state.user_id = ""
  ```

- This function allows switching between admin and user views
- When switching to user role, we reset the identification to force re-entry of email

### TODO 7: Implement admin interface

```python
def show_admin_interface(rag_manager):
    # TODO: Create admin title and info message
    # TODO: Implement file upload section
    # TODO: Handle file processing
   
    # TODO: Add knowledge base reset option
```

**Hints**:
- Create title and info with:
  ```python
  st.title("💰 Fintech Knowledge Administration")
  st.info("As an admin, you can upload financial knowledge that will be used to power the chatbot's responses.")
  ```

- File upload section:
  ```python
  st.subheader("Upload Knowledge File")
  uploaded_file = st.file_uploader("Choose a text file", type=["txt"], key="file_uploader")
  
  if uploaded_file:
      # Save the uploaded file to a temporary file
      with NamedTemporaryFile(delete=False, suffix=".txt") as tmp_file:
          tmp_file.write(uploaded_file.getvalue())
          temp_path = tmp_file.name
      
      # Process the file
      if st.button("Process File"):
          with st.spinner("Processing file..."):
              result = rag_manager.add_document_from_file(temp_path)
              st.success(result)
          
          # Clean up the temporary file
          os.unlink(temp_path)
  ```


  

### TODO 8: Implement user interface

```python
def show_user_interface(agent_executor_with_history):
    # TODO: Create user interface title
    # TODO: Handle email submission
    # TODO: Implement chat interface
    # TODO: Display sample questions in sidebar
```

**Hints**:
- Create title with `st.title("💰 Personal Finance Assistant")`

- Handle email submission:
  ```python
  if not st.session_state.email_submitted or not st.session_state.user_id:
      st.info("Please enter your email address to start chatting. This will be used to maintain your conversation history.")
      
      col1, col2 = st.columns([3, 1])
      with col1:
          email = st.text_input("Email Address:", key="email_input", placeholder="you@example.com")
      with col2:
          submit_button = st.button("Submit", type="primary")
          
      if submit_button:
          if submit_email(email):
              st.success("Email submitted successfully! You can now start chatting.")
              st.rerun()
          else:
              st.error("Please enter a valid email address.")
      
      # Show sample questions but disable chat until email is submitted
      st.sidebar.subheader("Sample Questions")
      st.sidebar.markdown("""
      - What's my current spending by category?
      - What's the price of AAPL stock?
      - How should I start investing with $1000?
      - Calculate a monthly mortgage payment for $300,000
      - What are the latest news about interest rates?
      - Show me my investment portfolio performance
      """)
      return
  ```

- Implement chat interface and sidebar (when email is submitted):
  ```python
  # Sidebar for conversation management
  with st.sidebar:
      st.subheader("Conversations")
      
      # New conversation button
      if st.button("New Conversation"):
          create_new_conversation()
          st.rerun()
      
      # List existing conversations for selection
      st.divider()
      for conv_id, conv_data in st.session_state.conversations.items():
          if st.button(f"Conversation {conv_data['number']}", key=conv_id):
              switch_conversation(conv_id)
              st.rerun()
  
  # Display conversation history
  for message in st.session_state.conversations[st.session_state.current_conversation_id]["messages"]:
      with st.chat_message(message["role"], avatar="💰" if message["role"] == "assistant" else None):
          st.markdown(message["content"])
  
  # Chat input
  if prompt := st.chat_input("Ask me anything about finance..."):
      with st.chat_message("user"):
          st.markdown(prompt)
      
      handle_chat_input(prompt, agent_executor_with_history)
  ```

- We show the email form until the user has submitted a valid email
- The sidebar allows users to create new conversations and switch between existing ones
- We display the conversation history for the current conversation
- The chat input is processed by the `handle_chat_input` function

### TODO 9: Implement email submission function

```python
def submit_email(email):
    # TODO: Validate email and set session state
```

**Hints**:
- Implement with:
  ```python
  def submit_email(email):
      if email and "@" in email and "." in email:  # Simple validation
          st.session_state.user_email = email
          st.session_state.user_id = email  # Use email as user ID
          st.session_state.email_submitted = True
          return True
      return False
  ```

- This function does a simple validation of the email format
- We use the email as both the user_email and user_id for simplicity
- Setting `email_submitted` to True will switch the UI to the chat interface

### TODO 10: Implement main function

```python
def main():
    # TODO: Set up components ( RAG manager, tools)
    # TODO: Set up agent
    # TODO: Create role selection in sidebar
    # TODO: Display appropriate interface based on role
```

**Hints**:
- Set up components:
  ```python
  # Set up components

  rag_manager = RAGManager()
  tools = setup_tools(rag_manager, llm, st.session_state.user_email)
  ```

- Set up agent:
  ```python
  # Set up agent
  agent_executor_with_history = setup_agent(tools)
  ```

- Create role selection:
  ```python
  # Role selection in sidebar
  with st.sidebar:
      st.header("Mode Selection")
      col1, col2 = st.columns(2)
      with col1:
          if st.button("User Mode", disabled=st.session_state.role=="user", use_container_width=True):
              set_role("user")
      with col2:
          if st.button("Admin Mode", disabled=st.session_state.role=="admin", use_container_width=True):
              set_role("admin")
  ```

- Display appropriate interface:
  ```python
  # Show interface based on role
  if st.session_state.role == "admin":
      show_admin_interface(rag_manager)
  else:
      show_user_interface(agent_executor_with_history)
  ```


- We pass the user's email to `setup_tools()` so it can be used in the tools
- The role selection buttons are disabled for the current role
- We display either the admin or user interface based on the current role

### TODO 11: Add entry point

```python
if __name__ == "__main__":
    main()
```

**Hints**:
- This is the standard Python idiom for making a script runnable
- When the script is run directly, it will call the main() function
- This allows the script to be imported without running the main function

## Running the Application

After completing all the above steps, you can run your application with this command:

```bash
streamlit run app.py
```

Your application should now be fully functional with these capabilities:

1. **Admin Interface**:
   - Upload financial knowledge via text files
   - Add knowledge directly through text input
   - Reset the knowledge base to default content

2. **User Interface**:
   - Email-based user identification
   - Multiple conversation management
   - Chat with the financial assistant
   - Get answers to financial questions using the various tools

## Understanding the Key Components

### 1. RAG System (Retrieval Augmented Generation)
- The RAG system stores financial knowledge in a vector database (Chroma)
- Documents are split into chunks and embedded using OpenAI embeddings
- When a query is made, the system retrieves relevant chunks and uses them to generate a response
- Conversation history is maintained to allow follow-up questions

### 2. Agent with Tools
- The agent can use multiple tools to answer user questions:
  - Stock price lookup: Get current stock prices
  - Financial knowledge retrieval: Access the RAG system
  - Python calculator: Perform calculations and analysis
  - Web search: Find current financial news and information
  - SQL database: Query user financial data

### 3. Conversation Management
- Each user has multiple conversations with separate histories
- Conversations are tracked using UUIDs and displayed with sequential numbers
- Messages are stored both for display and for conversation history retrieval

### 4. User Management
- Simple email-based user identification
- Database queries filter by user email to ensure privacy
- Admin and user roles with different interfaces

